In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unidecode
import time
from tqdm import tqdm
from IPython.display import Image, HTML


class csfd:

    def __init__(self,number_of_pages,imdb_rating = True):
        self.number_of_pages = number_of_pages
        self.imdb_rating = imdb_rating
        self.data = pd.DataFrame()
        self.allurl = []
        self.filmlinks = []
        self.headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"}

    def FilmLinks(self):
        
        for page in tqdm(range(1,1+self.number_of_pages),desc='Downloading film links from {} page(s)'.format(str(self.number_of_pages))):
            url = 'https://www.csfd.cz/podrobne-vyhledavani/strana-'+str(page)+'/?type%5B0%5D=0&genre%5Btype%5D=2&genre%5Binclude%5D%5B0%5D=&genre%5Bexclude%5D%5B0%5D=&origin%5Btype%5D=2&origin%5Binclude%5D%5B0%5D=&origin%5Bexclude%5D%5B0%5D=&year_from=&year_to=&rating_from=&rating_to=&actor=&director=&composer=&screenwriter=&author=&cinematographer=&production=&edit=&sound=&scenography=&mask=&costumes=&tag=&ok=Hledat&_form_=film'
            response = requests.get(url, headers = self.headers)
            souped = BeautifulSoup(response.text, 'html.parser')
            self.souped = souped
            self.allurl = [x.a.get('href') for x in self.souped.findAll('td', {'class':'name'})]
            self.filmlinks.extend(['https://www.csfd.cz' + str(x) for x in self.allurl])

        return(self.filmlinks)     
    
    


    def GetData(self):
        for filmurl in tqdm(self.FilmLinks(),desc='Downloading data for {} films'.format(str(len(self.filmlinks)))):
            response = requests.get(filmurl, headers = self.headers)
            souped = BeautifulSoup(response.text, 'html.parser')

            cztitle = souped.find('h1', {'itemprop':'name'}).string.replace("\t","").replace("\n","")

            try:
                entitle = souped.find('div',class_="info").find('img', alt='USA').findNext('h3').text
            except: 
                try:
                    entitle = souped.find('div',class_="info").find('img', alt='Velká Británie').findNext('h3').text
                except:
                    try:
                        entitle = souped.find('div',class_="info").find('img', alt='anglický').findNext('h3').text
                    except:
                        entitle = ''

            year = int(souped.find('span', itemprop='dateCreated').text)
            poster = souped.find(id='poster').img.get("src")
            genre = souped.find(class_='genre').text.split(' / ')
            origin = souped.find(class_='origin').text.split(', ')[0].split(' / ')
            length = int(souped.find(class_='origin').text.split(', ')[2].split(' ')[0])
            average = float(souped.find(class_='average').text.replace('%',''))
            count = int(souped.find(class_='count').text.replace('\nvšechna hodnocení(','').replace(')\n\t\t\t\t','').replace('\xa0',''))
            director = souped.find(itemprop="director").text.replace("\n","")

            data = pd.DataFrame({"poster":[poster],"cztitle":[cztitle],"entitle":[entitle],"year":[year],"genre":[genre],"origin":[origin],"length (min)":[length],"director":[director],"film_url":[filmurl],"CSFD avg rating (%)":[average],"CSFD num of ratings":[count]})

            if self.imdb_rating:
                try:
                    if entitle == '':
                        imdb = ImdbRating(cztitle,year).GetRating()
                    else:
                        imdb = ImdbRating(entitle,year).GetRating()
                    data = pd.concat([data, imdb], axis=1)
                except:
                    pass
            
            self.data = self.data.append(data,sort=False).reset_index(drop=True)
        
    def PrintData(self,n_rows=100):
        
        def path_to_image_html(path):
            return '<img src="'+ path + '"/>'
        
        return(HTML(self.data.to_html(escape=False,render_links=True,max_rows=n_rows,formatters=dict(poster=path_to_image_html))))